## objectives of pre-processing
    
- data should be 2D array
- input data has to be numbers
- no nan or inf
- coloum are scaled to similar ranges (mean=0, variance=1)
- coloums should not be coliner (cx1!=k*cx2)
- rows should not be causally dependent
- data should be 100 times larger then the number of coloums 

## import python libraries 

In [1]:
import os
import pathlib
from pprint import pprint
from pathlib import Path
import csv

In [2]:
import ray
import rasterio
import fiona
import pandas as pd
import xgboost as xgb
import numpy as np


from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
    

In [3]:
%load_ext memory_profiler

## define input datasets

- features 
- targets 
- out of sample 
- area of interest

In [4]:
%%time

feature_file_paths = [
Path("/g/data/ge3/sheece/LOC_distance_to_coast.tif"),
Path("/g/data/ge3/sheece/mrvbf_9.tif"),
Path("/g/data/ge3/sheece/relief_mrvbf_3s_mosaic.tif"),
Path("/g/data/ge3/sheece/relief_elev_focalrange1000m_3s.tif"),
Path("/g/data/ge3/sheece/relief_elev_focalrange300m_3s.tif"),
Path("/g/data/ge3/sheece/saga_wetSM_85_resampled.tif"),
Path("/g/data/ge3/sheece/tpi_300.tif"),
Path("/g/data/ge3/sheece/slope_fill2.tif"),
Path("/g/data/ge3/sheece/dem_fill.tif"),
Path("/g/data/ge3/sheece/3dem_mag2.tif"),
Path("/g/data/ge3/sheece/3dem_mag1_fin.tif"),
Path("/g/data/ge3/sheece/3dem_mag0.fin.tif"),
Path("/g/data/ge3/sheece/relief_roughness.tif"),
Path("/g/data/ge3/sheece/LATITUDE_GRID1_clip.tif"),
Path("/g/data/ge3/sheece/Dose_2016.tif"),
Path("/g/data/ge3/sheece/Potassium_2016.tif"),
Path("/g/data/ge3/sheece/Thorium_2016.tif"),
Path("/g/data/ge3/sheece/Rad2016U_Th.tif"),
Path("/g/data/ge3/sheece/Rad2016K_Th.tif"),
Path("/g/data/ge3/sheece/national_Wii_RF_multirandomforest_prediction.tif"),
Path("/g/data/ge3/sheece/si_geol1.tif"),
Path("/g/data/ge3/sheece/ceno_euc_aust1.tif"),
Path("/g/data/ge3/sheece/Grav_lane_clip.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-NIR-GREEN.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-RED-BLUE.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-SWIR1-SWIR2.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg_BLUE+SWIR2.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-SWIR1-NIR.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-CLAY-PC2.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-RED.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-GREEN.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-l8-all-85m-avg-BLUE.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-l8-all-85m-avg-NIR.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-SWIR1.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-SWIR2.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/s2-dpca-85m.tif"),
Path("/g/data/ge3/sheece/water-85m.tif"),
Path("/g/data/ge3/sheece/clim_EPA_albers.tif"),
Path("/g/data/ge3/sheece/Clim_Prescott_LindaGregory.tif"),
Path("/g/data/ge3/sheece/clim_PTA_albers.tif"),
Path("/g/data/ge3/sheece/clim_WDA_albers.tif"),
Path("/g/data/ge3/sheece/clim_RSM_albers.tif")
]

#target dataset small
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled_small.shp
#target dataset complete
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled.shp
target_file_path = Path("/g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled.shp")

# define a shape for area of intrest
area_of_interest_file_path = None

#OOS
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.shp
out_of_sample_file_path = Path("/g/data/ge3/sheece/0_50cm_2021_albers_C_oos.shp")

root = Path('/g/data/ge3/sheece')

CPU times: user 0 ns, sys: 724 µs, total: 724 µs
Wall time: 679 µs


In [8]:
if not os.path.isdir(root/"old"):
    os.mkdir(root/"old") 

## Standardising Datasets

In [12]:
# check if all datasets are in same projections 
import shutil
from rasterio.warp import calculate_default_transform, reproject, Resampling

# check projection 
crs_epsg3577 = rasterio.crs.CRS.from_string('EPSG:3577')

for feature_file_path in feature_file_paths:
    with rasterio.open(feature_file_path) as src:
        if crs_epsg3577 != src.crs:
                print("Converting dataset: "+str(feature_file_path))
                transform, width, height = calculate_default_transform(
                    src.crs, 
                    crs_epsg3577, 
                    src.width, 
                    src.height, 
                    *src.bounds)
                kwargs = src.meta.copy()
                kwargs.update({'crs': crs_epsg3577,'transform': transform, 'width': width,'height': height})
                
                
                with rasterio.open(root/ (feature_file_path.stem+'_resampled.tif'), 'w', **kwargs) as dst:
                    reproject(
                        source=rasterio.band(src, 1),
                        destination=rasterio.band(dst, 1),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=crs_epsg3577,
                        resampling=Resampling.nearest
                    )
                    print(dst.meta)
                    
                shutil.move(feature_file_path, root/("old/"+feature_file_path.name))
                output_path = root/ (feature_file_path.stem+'_resampled.tif')
                print(str(output_path))


In [13]:
# apply crop on target and feature datasets according to area of intrest 
import fiona
import rasterio
from rasterio.mask import mask

if area_of_interest_file_path is not None:
    with fiona.open(area_of_interest_file_path) as shapefile:
        geoms = [feature["geometry"] for feature in shapefile]

    for feature_file_path in feature_file_paths:
        if "cropped" not in feature_file_path.name:
            print("Cropping : "+feature_file_path.stem)
            # load the raster, mask it by the polygon and crop it
            with rasterio.open(feature_file_path) as src:
                out_image, out_transform = mask(src, geoms, crop=True)
            out_meta = src.meta.copy()

            # save the resulting raster  
            out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
            })   

            with rasterio.open(root/(feature_file_path.stem+'_cropped.tif'), "w", **out_meta) as dest:
                dest.write(out_image)

            shutil.move(feature_file_path, root/("old/"+feature_file_path.name))

In [14]:
# apply crop vector files(.shp)
import subprocess

if area_of_interest_file_path is not None:
    if "cropped" not in target_file_path.name:
        print("Cropping: "+target_file_path.name)
        clipped_file = root/(target_file_path.stem+'_cropped.shp')
        callstr = ['ogr2ogr',
                   '-clipsrc',
                   area_of_interest_file_path,
                   clipped_file,
                   target_file_path] 
        proc = subprocess.Popen(callstr, stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        stdout,stderr=proc.communicate()

        shutil.move(target_file_path, root/("old/"+target_file_path.name))

In [15]:
# create iterators ( C++ pointers) to inputs feature dataset and target dataset
datasets = []
input_feature_files = feature_file_paths
for feature_file_path in feature_file_paths: 
    datasets.append(rasterio.open(feature_file_path))

target_handle = fiona.open(target_file_path)

In [16]:
pprint(target_handle.meta)

{'crs': {'init': 'epsg:3577'},
 'crs_wkt': 'PROJCS["GDA94 / Australian '
            'Albers",GEOGCS["GDA94",DATUM["Geocentric_Datum_of_Australia_1994",SPHEROID["GRS '
            '1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6283"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4283"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",-18],PARAMETER["standard_parallel_2",-36],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",132],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3577"]]',
 'driver': 'ESRI Shapefile',
 'schema': {'geometry': 'Point',
            'properties': OrderedDict([('con', 'float:24.15')])}}


In [17]:
%%time
%%memit

head_row = ['target']
for input_feature_file in input_feature_files:
    head_row.append(input_feature_file.stem)

csv_rowlist = [head_row]
with open(root/'input_dataset.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for target in target_handle:
        value = target["properties"]["con"]
        x, y = target["geometry"]["coordinates"]
        new_row = [value]
        
        for dataset in datasets:
            new_row.append(next(dataset.sample([(x, y)]))[0])
        csv_rowlist.append(new_row)

        if len(csv_rowlist)%1000==0:
            writer.writerows(csv_rowlist)
            csv_rowlist = []

peak memory: 10982.55 MiB, increment: 10748.04 MiB
CPU times: user 19min 46s, sys: 2min 35s, total: 22min 21s
Wall time: 29min 51s


In [18]:
target_handle.close()
for dataset in datasets: 
    dataset.close()

In [47]:
%%time
%%memit

train = np.genfromtxt(str(root)+'/input_dataset.csv', delimiter=',')
train = train.astype(np.float32)
train = train[~np.isnan(train).any(axis=1)]
train = train[~(train == -9999.0).any(axis=1)]

with open(str(root)+'/input_dataset.csv', newline='') as f:
    reader = csv.reader(f)
    head_row = next(reader)
    
pd.DataFrame(train).to_csv(str(root)+'/input_dataset.csv',header=head_row, index=None)

peak memory: 12196.82 MiB, increment: 2280.30 MiB
CPU times: user 30 s, sys: 1.43 s, total: 31.4 s
Wall time: 31.6 s


In [63]:
with open(str(root)+'/input_dataset.csv', newline='') as f:
    reader = csv.reader(f)
    head_row = next(reader)
    
dtrain = xgb.DMatrix(str(root)+'/input_dataset.csv?format=csv&label_column=0',nthread=-1,feature_names=head_row[1:])


In [64]:
dtrain.num_row()

489318

In [57]:
dtrain.num_col()

42